In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from os.path import join as opj
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pylab
import os
from matplotlib import pyplot
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
from keras.optimizers import rmsprop
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping,ReduceLROnPlateau

from keras.datasets import cifar10
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
import xgboost as xgb
import lightgbm as lgb
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression as Log_Reg
from sklearn.metrics import log_loss

Using TensorFlow backend.
/share/apps/python3/3.5.3/intel/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train = pd.read_json("/".join(os.getcwd().split("/")[:-1])+"/Data/train.json")
test = pd.read_json("/".join(os.getcwd().split("/")[:-1])+"/Data/test.json")

In [3]:
y_train=train['is_iceberg']
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
train['inc_angle']=pd.to_numeric(train['inc_angle'], errors='coerce')#We have only 133 NAs.
train['inc_angle']=train['inc_angle'].fillna(method='pad')

X_angle=train['inc_angle']
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
X_test_angle=test['inc_angle']

#Generate the training data
X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
X_band_3=(X_band_1+X_band_2)/2
X_train = np.concatenate([X_band_1[:, :, :, np.newaxis], X_band_2[:, :, :, np.newaxis], X_band_3[:, :, :, np.newaxis]], axis=-1)



X_band_test_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
X_band_test_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
X_band_test_3=(X_band_test_1+X_band_test_2)/2
X_test = np.concatenate([X_band_test_1[:, :, :, np.newaxis], X_band_test_2[:, :, :, np.newaxis]], axis=-1)
X_test = np.concatenate([X_band_test_1[:, :, :, np.newaxis], X_band_test_2[:, :, :, np.newaxis], X_band_test_3[:, :, :, np.newaxis]], axis=-1)

In [4]:

#Img Generator
datagen = ImageDataGenerator(horizontal_flip = True,vertical_flip = True)
datagen.fit(X_train)
def gen_flow(X1, y,batch_size=64):
    genX1 = datagen.flow(X1,y,  batch_size=batch_size,seed=55)
    while True:
            X1i = genX1.next()
            yield X1i


In [5]:
#get early_stopping and best_weight_save
def get_callbacks(filepath, patience=13):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath,monitor='val_loss', save_best_only=True)
    reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, epsilon=1e-4, mode='min')
    return [es, msave,reduce_lr_loss]


In [6]:
#get vgg model
def VGG():
    base_model = VGG16(include_top=False, input_shape=X_train.shape[1:])
    x = base_model.get_layer('block5_pool').output
    
    x = GlobalMaxPooling2D(name = "glob_max_pool")(x)
    merge_one = Dense(128, activation='relu', name='fc2')(x)
    merge_one = Dropout(0.3)(merge_one)
    merge_one = Dense(32, activation='relu', name='fc3')(merge_one)
    merge_one = Dropout(0.3)(merge_one)
    
    predictions = Dense(1, activation='sigmoid')(merge_one)
    
    model = Model(input=base_model.input, output=predictions)   
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])
    return model


In [ ]:
def ensemble(vgg16_feature,train_idx,valid_idx,y_train, y_valid, X_test,train,valid,test):
    X_train = vgg16_feature[train_idx]
    X_valid = vgg16_feature[valid_idx]
    
    
    #XGboost
    XGB = xgb.XGBClassifier(reg_lambda = 1.6, max_depth=3, n_estimators=300, learning_rate=0.05,objective="binary:logistic").fit(X_train, y_train)
    train["Xgbosst"] = XGB.predict_proba(X_train)[:,1]
    valid.loc[valid_idx,"Xgbosst"] = XGB.predict_proba(X_valid)[:,1]
    test["Xgbosst"] = XGB.predict_proba(X_test)[:,1]
    print("XGboost train loss: "+str(log_loss(y_train,train["Xgbosst"])))
    print("XGboost validation loss: "+str(log_loss(y_valid,valid.loc[valid_idx,"Xgbosst"])))
    print()
 
    
    #svm
    svm_clf = svm.SVC(C= 0.6, probability = True).fit(X_train, y_train) 
    train["SVM"] = svm_clf.predict_proba(X_train)[:,1]
    valid.loc[valid_idx,"SVM"] = svm_clf.predict_proba(X_valid)[:,1]
    test["SVM"] = svm_clf.predict_proba(X_test)[:,1]
    print("SVM train loss: "+str(log_loss(y_train,train["SVM"])))
    print("SVM validation loss: "+str(log_loss(y_valid,valid.loc[valid_idx,"SVM"])))
    print()
    



    #RandomForestClassifier
    rf_clf = RandomForestClassifier(max_depth=3).fit(X_train, y_train)
    train["RandForest"] = rf_clf.predict_proba(X_train)[:,1]
    valid.loc[valid_idx,"RandForest"] = rf_clf.predict_proba(X_valid)[:,1]
    test["RandForest"] = rf_clf.predict_proba(X_test)[:,1]
    print("RandomForestClassifier train loss: "+str(log_loss(y_train,train["RandForest"])))
    print("RandomForestClassifier validation loss: "+str(log_loss(y_valid,valid.loc[valid_idx,"RandForest"])))
    print()



    
    return [train,valid,test]


In [ ]:
folds = list(StratifiedKFold(n_splits=3, shuffle=True, random_state=16).split(X_train, y_train))
valid = pd.DataFrame(index = [i for i in range(X_train.shape[0])])
test = pd.DataFrame()
log_reg = np.array([0.1]*len(X_train))

for j, (train_idx, valid_idx) in enumerate(folds):
    train = pd.DataFrame()
    X_train_cv = X_train[train_idx]
    y_train_cv = y_train[train_idx]
    X_valid_cv = X_train[valid_idx]
    y_valid_cv= y_train[valid_idx]


    #define file path and get callbacks
    file_path = "%s_aug_model_weights.hdf5"%j
    callbacks = get_callbacks(filepath=file_path, patience=5)
    gen_fl = gen_flow(X_train_cv,y_train_cv)
    vgg16 = VGG()

    vgg16.fit_generator(
            gen_fl,
            steps_per_epoch=24,
            epochs=100,
            shuffle=True,
            verbose=1,
            validation_data=(X_valid_cv, y_valid_cv),
            callbacks=callbacks)

    #Getting the Best Model
    vgg16.load_weights(filepath=file_path)

    #train predictions
    y_train_pred = vgg16.predict(X_train_cv)
    train['Vgg16'] = y_train_pred.reshape(y_train_pred.shape[0])

    #validation predictions
    y_valid_pred=vgg16.predict(X_valid_cv)
    valid.loc[valid_idx,'Vgg16'] = y_valid_pred.reshape(y_valid_pred.shape[0])

    #Getting Test predictions
    y_test_pred = vgg16.predict(X_test)
    test['Vgg16'] = y_test_pred.reshape(y_test_pred.shape[0])

    #ensemble
    intermediate_layer_model = Model(inputs=vgg16.input,outputs=vgg16.get_layer('glob_max_pool').output)
    vgg16_feature = intermediate_layer_model.predict(X_train);
    vgg16_test = intermediate_layer_model.predict(X_test)
    train,valid,test = ensemble(vgg16_feature,train_idx,valid_idx,y_train_cv, y_valid_cv, vgg16_test,train,valid,test);

    #logistic regression to assign weights
    logistic = Log_Reg(C = 0.6).fit(train,y_train_cv)
    log_train = logistic.predict_proba(train)
    log_reg[valid_idx] = logistic.predict_proba(valid.loc[valid_idx])[:,1]
    logistic_test = logistic.predict_proba(test)[:,1];

    print('Logistic Regression train loss: ' +str(log_loss(y_train_cv,log_train)))
    print('Logistic Regression validation loss: ' +str(log_loss(y_valid_cv,log_reg[valid_idx])))
    print()

    '''print("\ntrain score:")
    for clf in train.columns:
        print(clf+": "+str(log_loss(y_train_cv,train[clf])))
    print("Logistic_Reg: "+str(log_loss(y_train_cv,logistic_train)))'''
    break


print("\nValidation Results:\n")
valid['Logistic_Regression'] = log_reg
for clf in valid:
    print(clf+": "+str(log_loss(y_train,valid[clf])))



In [ ]:
def myAngleCV(X_train, X_test,y_train,K=3):
    folds = list(StratifiedKFold(n_splits=K, shuffle=True, random_state=16).split(X_train, y_train))
    valid = pd.DataFrame(index = [i for i in range(X_train.shape[0])])
    test = pd.DataFrame()
    log_reg = np.array([0]*len(X_train))
    
    for j, (train_idx, valid_idx) in enumerate(folds):
        train = pd.DataFrame()
        X_train_cv = X_train[train_idx]
        y_train_cv = y_train[train_idx]
        X_valid_cv = X_train[valid_idx]
        y_valid_cv= y_train[valid_idx]
        

        #define file path and get callbacks
        file_path = "%s_aug_model_weights.hdf5"%j
        callbacks = get_callbacks(filepath=file_path, patience=5)
        gen_fl = gen_flow(X_train_cv,y_train_cv)
        vgg16 = VGG()
        
        vgg16.fit_generator(
                gen_fl,
                steps_per_epoch=24,
                epochs=100,
                shuffle=True,
                verbose=1,
                validation_data=(X_valid_cv, y_valid_cv),
                callbacks=callbacks)

        #Getting the Best Model
        vgg16.load_weights(filepath=file_path)

        #train predictions
        y_train_pred = vgg16.predict(X_train_cv)
        train['Vgg16'] = y_train_pred.reshape(y_train_pred.shape[0])
        
        #validation predictions
        y_valid_pred=vgg16.predict(X_valid_cv)
        valid.loc[valid_idx,'Vgg16'] = y_valid_pred.reshape(y_valid_pred.shape[0])

        #Getting Test predictions
        y_test_pred = vgg16.predict(X_test)
        test['Vgg16'] = y_test_pred.reshape(y_test_pred.shape[0])
        
        #ensemble
        intermediate_layer_model = Model(inputs=vgg16.input,outputs=vgg16.get_layer('glob_max_pool').output)
        vgg16_feature = intermediate_layer_model.predict(X_train);
        vgg16_test = intermediate_layer_model.predict(X_test)
        train,valid,test = ensemble(vgg16_feature,train_idx,valid_idx,y_train_cv, y_valid_cv, vgg16_test,train,valid,test);
        
        #logistic regression to assign weights
        logistic = Log_Reg().fit(train,y_train_cv)
        logistic_train =  logistic.predict_proba(train)[:,1];
        log_reg[[valid_idx]] = logistic.predict_proba(valid.loc[valid_idx])[:,1];
        logistic_test = logistic.predict_proba(test)[:,1];
        for clf in train.columns:
            print(clf+": "+str(log_loss(y_train_cv,train[clf])))
        print("Logistic_Reg: "+str(log_loss(y_train_cv,logistic_train)))

    print("\n Validation Results\n:")
    valid['Logistic_Regression'] = log_reg
    for clf in valid:
        print(clf+": "+str(log_loss(y_train,valid[clf])))
    
    return valid,test

In [ ]:
n = 1

In [ ]:
tot = 0
for i in range(n):
    valid,submit=myAngleCV(X_train,X_test,y_train,K=3)
   

In [ ]:
print(n)